In [6]:
from pymongo import MongoClient
from pprint import pprint
import folium  

In [22]:
#2b  
client = MongoClient() # so we connect on the default host and port
db = client.fit3182_assignment_db # create a new database fit3182_assignment_db
the_collection = db.climate  # create a new collection called 'the_collection'
all_locations = the_collection.find({})
first_location = the_collection.find({}).limit(1)
for item in first_location:
    first_entry = item
m = folium.Map(location=[first_entry['latitude'], first_entry['longitude']])
for entries in all_locations:
    air_temperature = str(entries['air_temperature_celsius'])
    surface_temperature = str(entries['surface_temperature_celsius'])
    relative_humidity = str(entries['relative_humidity'])
    confidence = str(entries['confidence'])
    surface_string = 'Surface Temp' + ' ' + surface_temperature + '\n'  
    air_string = 'Air Temp' + ' ' + air_temperature + '\n'
    humidity = 'Humidity' + ' ' + relative_humidity + '\n'
    confid_string = 'Confidence' + ' ' + confidence
    full_string = surface_string + air_string + humidity + confid_string
    if entries['cause'] == 'natural':
        
        folium.Marker(
            location = [entries['latitude'], entries['longitude']],
            popup = full_string,
            icon = folium.Icon(color = "blue"),
        ).add_to(m)
    else:
        folium.Marker(
                location = [entries['latitude'], entries['longitude']],
                popup = full_string,
                icon = folium.Icon(color= "red"),
            ).add_to(m)     
        
m


